In [13]:
import numpy as np
def inv(α):
    involuta=np.tan(α)-α
    return involuta
# ENTRADAS
m=2
α=20                        # graus
ha=1.0                      # Jelaska
c=0.25                      # Jelaska
k=1                         # dente de altura completa usar 1.0 e dente diminuido usar 0,8 #SHIGLEY
i=2

#------------------------------------------- Cálculo Geometria --------------------------------------------
α=α*np.pi/180               # Conversão automática
r=0.38*m                    # Ver tooth profile geometry
z1_min=2*k*(m+(m**2+(1+2*m)*np.sin(α)**2)**0.5)/((1+2*m)*(np.sin(α))**2)
z1=int(z1_min+1)
x1=1-0.5*z1*(np.sin(α))**2
x2=-x1 # Ver cap 5 Arnaudov
z2=z1*i
inv_α=np.tan(α)-α
inv_αw12=2*np.tan(α)*(x1+x2)/(z1+z2)+inv_α # (Equação 5.1 Arnaudov)
αw12 = 0 # Operating pressure angle (chute inicial para o método numérico)
while (np.tan(αw12)- αw12) <inv_αw12:
    αw12=αw12+0.0001
    #print(αw12)
else:
    print("")
Δy12 = (x1+x2)-0.5*(z1+z2)*(np.cos(α)/np.cos(αw12)-1)# Return shift coefficient (tooth tip shortening coefficient) (Equação 5.4 Arnaudov)
ha1 = m*(ha+x1-Δy12) #Addendum  
ha2 = m*(ha+x2-Δy12) #Addendum
hf1 = m*(ha+x1+c) # dedendum
hf2 = m*(ha-x2+c) # dedendum
d1 = m*z1 #Reference circle diameters (Equação 5.8 Arnaudov)
d2 = m*z2 #Reference circle diameters (Equação 5.8 Arnaudov)
da1 = d1+ 2*ha1 #Tip circle diameters (Equação 5.9 Arnaudov)
da2 = d2+2*ha2 #Tip circle diameters (Equação 5.9 Arnaudov)
df1 = d1 - 2*hf1 #Root circle diameter (Equação 5.10 Arnaudov)
df2 = d2 - 2*hf2 #Root circle diameter (Equação 5.10 Arnaudov)

db1 = d1*np.cos(α)
db2 = d2*np.cos(α)
αa1=np.arccos(db1/da1) # Tip circle profile angle
αa2=np.arccos(db2/da2) # Tip circle profile angle
ψ1= np.pi/(2*z1)+2*x1*np.tan(α)/z1 #equação 2.73 Jelaska # ângulo entre o centro do dente até a involuta no ponto em que cruza o diâmetro primitivo
ψ2= np.pi/(2*z2)+2*x2*np.tan(α)/z2 #equação 2.73 Jelaska
print("SAÍDAS:")
print("inv(α):",inv(α)*180/np.pi,"deg")
print("ψ1:",180*ψ1/np.pi,"deg")
print("ψ2:",180*ψ2/np.pi,"deg")
print("inv(αa1):",inv(αa1)*180/np.pi,"deg")
print("inv(αa2):",inv(αa2)*180/np.pi,"deg")
print("d1:",d1,"mm","\ndb1:",db1,"mm","\ndf1:",df1,"mm","\nda1:",da1,"mm")
print("d2:",d2,"mm","\ndb2:",db2,"mm","\ndf2:",df2,"mm","\nda2:",da2,"mm")
print("r:",r,"mm")


SAÍDAS:
inv(α): 0.8539582918412502 deg
ψ1: 6.341078059574837 deg
ψ2: 2.8294609702125806 deg
inv(αa1): 5.209137070714542 deg
inv(αa2): 2.3014180768603967 deg
d1: 30 mm 
db1: 28.190778623577252 mm 
df1: 24.50933335321533 mm 
da1: 34.49178536261742 mm
d2: 60 mm 
db2: 56.381557247154504 mm 
df2: 54.509333353215325 mm 
da2: 63.51045206904808 mm
r: 0.76 mm


In [14]:
B=5 # Espessura dos dentes das engrenagens
T=6 # Torque nominal do motor IMOBRAS
n=82 # rotação nominal do motor
γFS1 = 4.45 # Fator de forma é definido graficamente em função de f(Z1,x1)  pag 165 arnaudov 
γFS2 = 4.22 # Fator de forma é definido graficamente em função de f(Z2,x2)  pag 165 arnaudov 
Ka=1.25 # Fator de carga, Tabela 17.1 pag 163 arnaudov
Kα =1.2 # Fator de carga transversaNo hardened gear (HB>350)
Kγmax = 1.05 # Fator de carga interno Tabela 17.2 arnaudov
ZH12 = 2.5 # Fator de zona f(x1,x2,z1,z2), quando (x1+x2)/(z1+z2)=0 Pag 166 Arnaudov 
σFG1 = 379 # Resistência à tensão de flexão (bending strength).
σHG1 = 1241 # Resistência à tensão de contato (contact strength).
ZE12 = 190 # Fator elástico (Quando as duas engrenagens são de aço e usinadas ZE=190)

In [15]:
Ft=2000*T/d1
v=np.pi*d1*n/60000
εα12 = z1*(np.tan(αa1)-np.tan(αw12))/(2*np.pi)+z2*(np.tan(αa2)-np.tan(αw12))/(2*np.pi) # equação  11.3 Arnaudov
γε12 = 0.25+0.75/εα12
Kv12=1+(25*B/(Ft*Ka)+0.002)*(z1*v/100)*((i**2)/(i**2+1))**0.5
Kβ12=1+0.5*(B/d1)**2
σF1=γFS1*γε12*Ka*Kv12*Kβ12*Kα*Kγmax*Ft/(B*m)
Zε12 = ((4-εα12)/3)**0.5
σH12 = ZH12*ZE12*Zε12*((Ft/(B*d1))*((i+1)/i)*Ka*Kv12*Kβ12*Kα*Kγmax)**0.5
SH1=σHG1/σH12
SF1=σFG1/σF1
SF1,SH1

(1.810353385977036, 1.1418297292918607)